# [모듈 3.5] 엔드포인트 배포 스텝 개발 (SageMaker Model Building Pipeline 배포 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 10분 소요 됩니다.

- 1. 모델 엔드포인트 배포 개요
- 2. 기본 라이브러리 로딩
- 3. 모델 빌딩 파이프라인 의 스텝(Step) 생성
- 4. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 5. 세이지 메이커 스튜디오에서 확인하기
- 6. 앤드포인트 추론 테스트
- 7. 엔드포인트 삭제


# 1. 모델 엔드포인트 배포 개요 

---
## 1.1. SageMaker 호스팅 아키텍쳐
- 일반적인 아키텍쳐의 그림이고, 오토 스케일링이 적용이 되어 있습니다.

![sagemaker-endpoints.png](img/sagemaker-endpoints.png)

## 1.2 프로세싱 스텝 
- 엔드포인트 배포는 "deploy_model.py" 의 스크립트를 실행하여 배포를 합니다. 그래서 여기서는 "프로세싱 스텝" 을 사용합니다.
- 프로세싱 단계의 개발자 가이드 
    - [프로세싱 스텝](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)



# 2. 기본 라이브러리 로딩

세이지 메이커 관련 라이브러리를 로딩 합니다.

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()
%store -r 

%store

Stored variables and their in-db values:
bucket                                 -> 'sagemaker-us-east-1-585843180719'
claims_data_uri                        -> 's3://sagemaker-us-east-1-585843180719/sagemaker-w
customers_data_uri                     -> 's3://sagemaker-us-east-1-585843180719/sagemaker-w
hyperparameters                        -> {'scale_pos_weight': '29', 'max_depth': '3', 'eta'
image_uri                              -> '683313688378.dkr.ecr.us-east-1.amazonaws.com/sage
input_data_uri                         -> 's3://sagemaker-us-east-1-585843180719/sagemaker-w
preprocessing_code                     -> 'src/preprocessing.py'
project_prefix                         -> 'sagemaker-webinar-pipeline-base'
sagemaker_model                        -> 'pipelines-ch0l86ldquad-fraud-basic-createmo-lyzrs
test_preproc_dir_artifact              -> 's3://sagemaker-us-east-1-585843180719/sagemaker-w
train_model_artifact                   -> 's3://sagemaker-us-east-1-585843180719/sagemaker

# 3. 모델 빌딩 파이프라인 의 스텝(Step) 생성


## 3.1 모델 빌딩 파이프라인 변수 생성


In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)
endpoint_instance_type = ParameterString(
    name="EndpointInstanceType",
    default_value="ml.m5.xlarge"
)

## 3.2 배포에 사용할 스크립트 코드 S3 업로딩


In [3]:
from datetime import datetime
suffix = datetime.now().microsecond
print("suffix: ", suffix)


local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: \n", s3_deploy_code_uri)

pipeline_endpoint_name = 'pipeline-endpoint-'  + str(suffix)

suffix:  460471
s3_deploy_code_uri: 
 s3://sagemaker-us-east-1-585843180719/sagemaker-webinar-pipeline-base/code/deploy_model.py


## 3.3. 배포에 사용할 프로세서 정의

In [4]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role= role,
    instance_type= processing_instance_type, 
    instance_count= processing_instance_count,
    base_job_name='fraud-scratch-deploy-model',
    sagemaker_session=sagemaker_session)




The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


## 3.4 모델 엔트포인트 생성 스탭 생성


In [5]:
deploy_step = ProcessingStep(
    name='Fraud-Basic-Endpoint',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", sagemaker_model, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

# 4. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행



## 4.1 파이프라인 정의


[중요] `pipeline_endpoint_name` 에 '_' 언데바를 넣으면 에러가 납니다. '-' 대시는 가능합니다.

In [6]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        endpoint_instance_type
    ],
    steps=[deploy_step],
)

## 4.2 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [7]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'EndpointInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Fraud-Basic-Endpoint',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3',
     'ContainerArguments': ['--model_name',
      'pipelines-ch0l86ldquad-fraud-basic-createmo-lyzrspepk1',
      '--region',


## 4.3 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [8]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [9]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:585843180719:pipeline/sagemaker-webinar-pipeline-base',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:585843180719:pipeline/sagemaker-webinar-pipeline-base/execution/g2mrqerf6oje',
 'PipelineExecutionDisplayName': 'execution-1678716430105',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 3, 13, 14, 7, 10, 28000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 3, 13, 14, 7, 10, 28000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '77da98e7-b9fc-4d9d-81b8-a24fc6deaddd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '77da98e7-b9fc-4d9d-81b8-a24fc6deaddd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '427',
   'date': 'Mon, 13 Mar 2023 14:07:10 GMT'},
  'RetryAttempts': 0}}

## 4.4 파이프라인 실행 기다리기

In [10]:
execution.wait()

## 4.5 파이프라인 실행 단계 기록 보기

In [11]:
execution.list_steps()

[{'StepName': 'Fraud-Basic-Endpoint',
  'StartTime': datetime.datetime(2023, 3, 13, 14, 7, 11, 352000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 3, 13, 14, 13, 48, 236000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:585843180719:processing-job/pipelines-g2mrqerf6oje-fraud-basic-endpoint-le0j0qqqsl'}}}]

# 5. 세이지 메이커 스튜디오에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
    - SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)



![endpoint-sm.png](img/endpoint-sm.png)

# 6. 앤드포인트 추론 테스트

### 추론에 사용할 Payload 생성 및 추론
- 테스트 데이터를 통해서 엔드포인트에 전달할 CSV 형태의 String을 생성합니다. (payload).
- payload 를 엔드포인트에 제공하면, 확률값을 0.072 을 리턴합니다. 
- 보통 확률값이 0.5 보다 작으면 0 (Non-Fruad), 0.5 보다 크면 1 (Fruad) 로 변환해서 예측값으로 사용합니다.

```
payload: 
 6038.102399076349,15838.10239907635,39,64,0,1,750,2750,95660,2012,1,0,0,9800,1,9,2,16,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0
Reponse: 
[['0.0726071447134018']]


```

## 6.1. 테스트 데이터 준비

In [12]:
import boto3
sagemaker_boto_client = boto3.client('sagemaker')
from src.p_utils import get_predictor, get_payload, predict

from sagemaker.deserializers import CSVDeserializer
csv_deserializer = CSVDeserializer(accept='text/csv') # 디폴트가 accept가 'text/csv' 이지만 직관적 이유로 기술함.



In [13]:
test_preproc_path = f"{test_preproc_dir_artifact}/test.csv"
test_df = pd.read_csv(test_preproc_path)
test_df.head(1)


fraud  vehicle_claim  total_claim_amount  customer_age  months_as_customer  \
0      0   23901.432507        36201.432507            56                 118   

   num_claims_past_year  num_insurers_past_5_years  policy_deductable  \
0                     0                          1                750   

   policy_annual_premium  customer_zip  ...  collision_type_missing  \
0                   3000         98039  ...                       0   

   incident_severity_Major  incident_severity_Minor  \
0                        0                        0   

   incident_severity_Totaled  authorities_contacted_Ambulance  \
0                          1                                0   

   authorities_contacted_Fire  authorities_contacted_None  \
0                           0                           0   

   authorities_contacted_Police  police_report_available_No  \
0                             1                           0   

   police_report_available_Yes  
0                            1  

[1 rows x 59 columns]

## 6.2. 엔드포인트에 입력이 될 데이터 (payload) 생성

In [14]:
# test_df 의 상위 1개 레코드를 사용
payload = get_payload(test_df, label_col = 'fraud', verbose=False)
print("payload: \n", payload)



payload: 
 23901.43250714285,36201.43250714285,56,118,0,1,750,3000,98039,2012,3,3,1,12300,9,12,3,14,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1


## 6.3. 추론 하여 예측 확률 얻기

In [15]:

predictor = get_predictor(pipeline_endpoint_name, sagemaker_session, csv_deserializer)

pred_prob = predict(predictor, payload)
print("prediction_prob: \n", pred_prob)



prediction_prob: 
 ['0.10092484205961227']


# 7. 엔드포인트 삭제

파이프라인 실행을 통하여 생성된 앤드포인트 및 관련 오브젝트 (세이지 메이커 모델, 엔드포인트 컨피그) 를 삭제 합니다.

In [16]:
from src.p_utils import delete_endpoint

delete_endpoint(sagemaker_boto_client, pipeline_endpoint_name, is_del_model = True )

--- Deleted model: pipelines-ch0l86ldquad-fraud-basic-createmo-lyzrspepk1
--- Deleted endpoint: pipeline-endpoint-460471
--- Deleted endpoint_config: pipelines-ch0l86ldquad-fraud-basic-createmo-lyzrspepk1-config
